In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, callbacks

In [7]:
data_date = "2023-06-25"

In [8]:
df = pd.read_csv(data_date + "_data_cleaned_location_enhanced.csv")

In [9]:
# Drop the specified columns
df = df.drop(columns=["url_id", "Latitude", "Longitude", "Region"])

# Check for missing values
missing_values = df.isnull().sum().sum()

# Separate the target variable
y = df.pop("Celková cena")
X = df

# Apply normalization
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

missing_values


0

In [10]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.20, random_state=42)

X_train.shape, X_test.shape


((12543, 170), (3136, 170))

In [12]:
#Model Architecture
model = models.Sequential([
    layers.Dense(512, activation='relu', kernel_initializer='he_normal', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    layers.Dense(1)
])

# Reduced Learning Rate
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[5000, 10000],
    values=[1e-3, 5e-4, 2e-4])
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

# Callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train
history = model.fit(X_train, y_train, epochs=250, batch_size=32, validation_split=0.2, verbose=1, 
                    callbacks=[reduce_lr, checkpoint, early_stopping])


Epoch 1/250
314/314 [==============================] - 1s 3ms/step - loss: 7836566290432.0000 - mean_absolute_error: 2110209.7500 - val_loss: 3088913268736.0000 - val_mean_absolute_error: 1396788.0000 - lr: 0.0010
Epoch 2/250
314/314 [==============================] - 1s 2ms/step - loss: 2816561905664.0000 - mean_absolute_error: 1298637.2500 - val_loss: 2399180685312.0000 - val_mean_absolute_error: 1191985.7500 - lr: 0.0010
Epoch 3/250
314/314 [==============================] - 1s 2ms/step - loss: 2245089034240.0000 - mean_absolute_error: 1132512.2500 - val_loss: 1958390136832.0000 - val_mean_absolute_error: 1082341.3750 - lr: 0.0010
Epoch 4/250
314/314 [==============================] - 1s 2ms/step - loss: 1696819838976.0000 - mean_absolute_error: 962440.1875 - val_loss: 1367145840640.0000 - val_mean_absolute_error: 852823.2500 - lr: 0.0010
Epoch 5/250
314/314 [==============================] - 1s 2ms/step - loss: 1234851201024.0000 - mean_absolute_error: 797781.4375 - val_loss: 10249